# Generate points for the torus and confused circle

In [4]:
using Revise

In [5]:
includet("../../../extension_method.jl")
using .ext

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-13774971255847585214\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13779192412089632505\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-13774971255847585214\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


In [6]:
using Distributions
using Eirene
using Plots
using JLD
plotlyjs()

Plots.PlotlyJSBackend()

#### Sample from torus

In [55]:
# generate points on torus
n_torus = 300
R = 2
r = 1
torus, torus_theta, torus_phi = sample_torus(R, r, n_torus, return_angles = true);

#### Sample from circle

For confused circle, first decide on the relationship between $\theta$ and $\phi$

In [56]:
function theta_from_phi(phi, h, a)
    # adjust maximum height (h) and range (π-a, π+a)  
    
    if phi <= π - a
        rand(Uniform(0, 0.25))
    elseif π - a < phi <= π 
        h *(phi .- (π .- a)) / a 
    elseif π < phi <= π + a
        h .- h * (phi .- π ) / a
    else
        rand(Uniform(0, 0.25))
    end
end

theta_from_phi (generic function with 1 method)

In [57]:
h = 1.5* π
a = 0.5
function theta_function(phi)
   theta_from_phi(phi, h, a) 
end

theta_function (generic function with 1 method)

In [58]:
# visualize theta_function
plot(range(0, stop = 2π, length = 1000), theta_function, xlabel ="ϕ", ylabel ="θ", label="")

<!DOCTYPE html>
 
 
 Plots.jl

In [59]:
##### generate points on a "confused circle"

# sample phi for the "circular" section and the "confused" section
n_circular = 40 
n_confused = 60
phi = rand(Uniform(0, 2π), n_circular)
phi_confused = rand(Uniform(π-a, π+a), n_confused)
circle_phi = vcat(phi, phi_confused)

# find theta
circle_theta = [theta_function(item) for item in circle_phi]

# coordinates in 3D 
x = (R .+ r .* cos.(circle_theta .+ pi)).* cos.(circle_phi)
y = (R .+ r .* cos.(circle_theta .+ pi)) .* sin.(circle_phi)
z = r .* sin.(circle_theta .+ pi)

circle = cat(x,y,z, dims = 2);

In [60]:
p = plot_3D(circle, torus)
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl

Plot the points on square. Assume that the sides of the squares have been identified to represent a torus

In [61]:
# plot angles
# Assign colors to visualization
P_color = "#ff8d00"
Q_color = "#008181";

plot(framestyle = :box, yaxis = nothing, xaxis = nothing, legend = :false, size = (400, 400))
# plot torus
scatter!(torus_theta, torus_phi, color = Q_color, markersize = 5, label = "Q")

# plot circle
scatter!(circle_theta, circle_phi, color = P_color, markersize = 7, marker =:xcross, label = "P")

<!DOCTYPE html>
 
 
 Plots.jl

# Run VR and Witness persistence 
To check that the barcodes are consistent with expection

In [62]:
# compute distance
D_torus, D_circle, D_torus_circle, D_circle_torus = compute_distance(torus, circle);

In [63]:
# Compute Vietoris-Rips persistence on two regions
dim = 1
VR_torus = eirene(D_torus, record = "all", maxdim = dim)
VR_circle = eirene(D_circle, record = "all", maxdim = dim);

In [64]:
p1 = plot_barcode(barcode(VR_torus, dim = 1), lw = 3, title = "Barcode(VR(torus))", titlefontsize = 12)
p2 = plot_barcode(barcode(VR_circle, dim = 1), lw = 2, title = "Barcode(VR(circle))", titlefontsize = 12)
plot(p1, p2, layout = grid(2,1), size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

In [65]:
torus_2d = hcat(torus_theta, torus_phi)
circle_2d = hcat(circle_theta, circle_phi);

In [66]:
# plot cycle rep of selected bar
VR_P_class = 6
cycle = classrep(VR_circle, class = VR_P_class, dim = 1)
cycle = [sort(cycle[:,i]) for i = 1:size(cycle,2)]
plot_cycle_square_torus(circle_2d, torus_2d, cycle = cycle)

<!DOCTYPE html>
 
 
 Plots.jl

In [67]:
# compute Witness persistence
W_torus_circle = compute_Witness_persistence(D_torus_circle, maxdim = dim)
W_circle_torus = compute_Witness_persistence(D_circle_torus, maxdim = dim)

Dict{Any,Any} with 8 entries:
  "fv"                         => Any[Any[0.128008, 0.150577, 0.311159, 0.18412…
  "param_max"                  => 2.02191
  "default_vertex_to_W_vertex" => nothing
  "simplex2index"              => Dict([26, 90, 95]=>96826,[34, 98]=>2803,[7, 1…
  "eirene_output"              => Dict{String,Any}("rv"=>Any[Int64[], [1, 2, 1,…
  "index2simplex"              => Dict((115900, 2)=>[34, 91, 96],(151029, 2)=>[…
  "distance_matrix"            => [3.01465 3.89825 … 1.97031 1.797; 1.40954 2.0…
  "W_vertex_to_default_vertex" => nothing

In [68]:
# plot all four barcodes
l = grid(4,1)
p1 = plot_barcode(barcode(VR_torus, dim = 1), lw = 2, title = "Barcode(VR(torus))", titlefontsize = 12)
p2 = plot_barcode(barcode(W_torus_circle["eirene_output"], dim = 1) , lw = 5, title = "Barcode(W(torus, circle))", titlefontsize = 12)
p3 = plot_barcode(barcode(W_circle_torus["eirene_output"], dim = 1) , lw = 5, title = "Barcode(W(circle, torus))", titlefontsize = 12)
p4 = plot_barcode(barcode(VR_circle, dim = 1), lw = 5, title = "Barcode(VR(circle))", titlefontsize = 12)
plot(p1, p2, p3, p4, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

In [69]:
# save points
#save("data.jld", "torus", torus, "torus_theta", torus_theta, "torus_phi", torus_phi,
#    "circle", circle, "circle_theta", circle_theta, "circle_phi", circle_phi)

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215
